In [ ]:
### https://www.youtube.com/watch?v=njE89SwtPYI&t=02h33m33s

In [ ]:
#
# Set-ExecutionPolicy Bypass -Scope Process
#
import pyspark
from delta import configure_spark_with_delta_pip
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#
#
#### _spark = SparkSession.builder.master('local[1]').getOrCreate()
#
#
#				.config(	'spark.sql.catalog.spark_catalog'
#						,	'io.unitycatalog.spark.UCSingleCatalog') \
#				.config(	'spark.sql.catalog.unity'
#						,	'io.unitycatalog.spark.UCSingleCatalog') \
#				.config(	'spark.sql.defaultCatalog'
#						,	'unity') \
#				.config(	'spark.sql.catalog.unity.uri'
#						,	'http://localhost:4041') \
#
_builder =	(	SparkSession.builder.master('local[1]') \
				.appName('pyspark-deltalake-local-testing') \
				.config(	'spark.sql.extensions'
						,	'io.delta.sql.DeltaSparkSessionExtension')
				.config(	'spark.sql.catalog.spark_catalog'
						,	'org.apache.spark.sql.delta.catalog.DeltaCatalog'))
#
_spark = configure_spark_with_delta_pip(_builder).enableHiveSupport().getOrCreate()
#
#
#
### READ
_demoDataFrame = _spark.read \
	.option('inferSchema', True) \
	.option('header',True) \
	.format('parquet') \
	.load('file:///C:/Users/Administrator/Documents/Code/Python/PySpark/demo.parquet')
#
_demoDataFrame.show(10, truncate=False)
#

+---------+---------+--------+--------+----------+-------+---+-----+----+-------------+------------------------------------+--------------+----------+
|Branch_ID|Dealer_ID|Model_ID|Revenue |Units_Sold|Date_ID|Day|Month|Year|BranchName   |DealerName                          |model_category|RevPerUnit|
+---------+---------+--------+--------+----------+-------+---+-----+----+-------------+------------------------------------+--------------+----------+
|BR0074   |DLR0215  |Toy-M195|3163004 |2         |DT00045|19 |1    |2017|Anadol Motors|Wanderer Motors                     |Toy           |1581502.0 |
|BR0075   |DLR0195  |Mah-M165|21878175|3         |DT00046|23 |9    |2017|Anadol Motors|Tornado Motors                      |Mah           |7292725.0 |
|BR0077   |DLR0155  |Toy-M105|9867107 |1         |DT00046|31 |8    |2017|Anadol Motors|Piaggio Motors                      |Toy           |9867107.0 |
|BR0078   |DLR0135  |Hyu-M75 |4583883 |3         |DT00047|5  |5    |2017|Anadol Motors|Morgan 

In [2]:
_spark

In [ ]:
#
## ## drop if require to delete from catalog
#_query = 'DROP TABLE CarDealers'
#_spark.sql(_query)
#

DataFrame[]

In [4]:
#
### WRITE as TABLE-DELTA
#
_path = 'file:///C:/Users/Administrator/Documents/Code/Python/PySpark/MyDeltaLake/CarDealers/'
#
_demoDataFrame.write \
				.mode('overwrite') \
				.format('delta') \
				.option('path',_path) \
				.saveAsTable(	\
								'CarDealers'	\
							,	path	=	_path
							,	format	=	'delta'	\
							,	mode	=	'overwrite')
#
"""
%sql
CREATE TABLE db_catalog_name.db_catalog_schema_name.CarDealers
USING DELTA
LOCATION 'abfss://destination_name@db_locationDataLake_name.dbfs.core.windows.net/parquetDataFolderName'
"""
#

"\n%sql\nCREATE TABLE db_catalog_name.db_catalog_schema_name.CarDealers\nUSING DELTA\nLOCATION 'abfss://destination_name@db_locationDataLake_name.dbfs.core.windows.net/parquetDataFolderName'\n"

In [5]:
#
_resultsAsSQL_DataFrame = _spark.sql('DESCRIBE EXTENDED CarDealers')
#
_resultsAsSQL_DataFrame.show(truncate=False)
#

+----------------------------+---------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                        |comment|
+----------------------------+---------------------------------------------------------------------------------+-------+
|Branch_ID                   |string                                                                           |NULL   |
|Dealer_ID                   |string                                                                           |NULL   |
|Model_ID                    |string                                                                           |NULL   |
|Revenue                     |bigint                                                                           |NULL   |
|Units_Sold                  |bigint                                                                           |NULL   |
|Date_ID                     |st

In [9]:
#
_resultsAsSQL_DataFrame = _spark.sql('DESCRIBE DETAIL CarDealers')
#
_resultsAsSQL_DataFrame.show(truncate=False)
#

+------+------------------------------------+--------------------------------+-----------+---------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                            |description|location                                                                         |createdAt              |lastModified           |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+--------------------------------+-----------+---------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+-----------------+--------+-----------+----------+----------------+--

In [6]:
#
_resultsAsSQL_DataFrame = _spark.sql('SELECT * FROM CarDealers ORDER BY Year DESC, Month DESC, Day DESC;')
#
_resultsAsSQL_DataFrame.show(10, truncate=False)
#

+---------+---------+--------+--------+----------+-------+---+-----+----+------------------------------------+------------------+--------------+----------+
|Branch_ID|Dealer_ID|Model_ID|Revenue |Units_Sold|Date_ID|Day|Month|Year|BranchName                          |DealerName        |model_category|RevPerUnit|
+---------+---------+--------+--------+----------+-------+---+-----+----+------------------------------------+------------------+--------------+----------+
|BR2353   |DLR0196  |Mah-M176|7236359 |1         |DT01136|31 |12   |2020|Cadillac Automobile Company Building|Toyota Motors     |Mah           |7236359.0 |
|BR2468   |DLR0107  |Cad-M37 |3108616 |1         |DT01185|30 |12   |2020|Kindel Building                     |Land Rover Motors |Cad           |3108616.0 |
|BR2323   |DLR0193  |Mah-M173|6271301 |1         |DT01119|28 |12   |2020|Blankinship Motor Company Building  |Tazzari Motors    |Mah           |6271301.0 |
|BR2448   |DLR0105  |Cad-M35 |4116309 |3         |DT01179|28 |12

In [7]:
_spark.catalog.getTable('CarDealers')

Table(name='CarDealers', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False)

In [8]:
_spark.catalog.currentCatalog()

'spark_catalog'

In [9]:
_spark.catalog.getDatabase('default')

Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/C:/Users/Administrator/Documents/Code/Python/PySpark/spark-warehouse')

In [10]:
_spark.catalog.getDatabase('default').locationUri

'file:/C:/Users/Administrator/Documents/Code/Python/PySpark/spark-warehouse'

### HOW QUERY DIRECTLY FROM DELTA FOLDER

In [11]:
#
_resultsAsSQL_DataFrame = _spark.sql('SELECT * FROM delta.`file:/C:/Users/Administrator/Documents/Code/Python/PySpark/MyDeltaLake/CarDealers`;')
#
_resultsAsSQL_DataFrame.show(10, truncate=False)
#

+---------+---------+--------+--------+----------+-------+---+-----+----+-------------+------------------------------------+--------------+----------+
|Branch_ID|Dealer_ID|Model_ID|Revenue |Units_Sold|Date_ID|Day|Month|Year|BranchName   |DealerName                          |model_category|RevPerUnit|
+---------+---------+--------+--------+----------+-------+---+-----+----+-------------+------------------------------------+--------------+----------+
|BR0074   |DLR0215  |Toy-M195|3163004 |2         |DT00045|19 |1    |2017|Anadol Motors|Wanderer Motors                     |Toy           |1581502.0 |
|BR0075   |DLR0195  |Mah-M165|21878175|3         |DT00046|23 |9    |2017|Anadol Motors|Tornado Motors                      |Mah           |7292725.0 |
|BR0077   |DLR0155  |Toy-M105|9867107 |1         |DT00046|31 |8    |2017|Anadol Motors|Piaggio Motors                      |Toy           |9867107.0 |
|BR0078   |DLR0135  |Hyu-M75 |4583883 |3         |DT00047|5  |5    |2017|Anadol Motors|Morgan 

# Unity Metastore (ADLS)
# |
## Catalog
# |
### DataBase
# |
#### Table

<!-- ![idempotency](https://thetightlycoupled.wordpress.com/wp-content/uploads/2020/02/idempotent.png "idempotency") -->
<img src="./resources/idempotent.png" alt="idempotency" width="400">

<br>

<img src="./resources/612ee17c-c597-4f53-bb3a-67eafb5f0e17_1080x994.png" alt="idempotency" height="500">
<!-- ![idempotency](./resources/612ee17c-c597-4f53-bb3a-67eafb5f0e17_1080x994.png "idempotency") -->

##
#### REF : https://medium.com/@riyukhandelwal/auto-loader-handling-incremental-etl-with-databricks-ae71687a281b
##
#### https://www.youtube.com/watch?v=njE89SwtPYI&t=02h33m33s
##

# Incrementally Loading Files:

```python
#
## .option('cloudFiles.schemaEvolutionMode','addNewColumns') <<<<< this is the default, which means that if you don't included, will take it exactly the same
#
## ref : https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/schema
## ref : https://medium.com/@nivethanvenkat28/spark-structured-streaming-for-incremental-batch-processing-using-databricks-autoloader-48316b36973b
#
_autoLoadedDataFrameFromFile = _spark.readStream \
	.format('cloudFiles') \
	.option('cloudFiles.format','csv') \
	.option('cloudFiles.schemaLocation','file:///C:/Users/Administrator/Documents/Code/Python/PySpark/Autoloaded/Checkpoint/') \
	.option('cloudFiles.schemaEvolutionMode','addNewColumns') \
	.option('schemaHints',True) \
	.load('file:///C:/Users/Administrator/Documents/Code/Python/PySpark/Autoloaded/Raw/')
#
```

Auto Loader supports the following modes for schema evolution, which you set in the option `cloudFiles.schemaEvolutionMode`:

| Mode                     | Behavior on reading new column                                                                                                                                 |
|--------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `addNewColumns` (default) | Stream fails. New columns are added to the schema. Existing columns do not evolve data types.                                                                 |
| `rescue`                 | Schema is never evolved and <font color='red'>stream does not fail due to schema changes</font>. All new columns are recorded in the **rescued data column**.                    |
| `failOnNewColumns`       | Stream fails. Stream does not restart unless the provided schema is updated, or the offending data file is removed.                                            |
| `none`                   | Does not evolve the schema, new columns are ignored, and data is not rescued unless the `rescuedDataColumn` option is set. Stream does not fail due to schema changes. |

REF : https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/schema

```python
#
_autoLoadedDataFrameFromFile.writeStream.format('delta) \
	.option('checkpointLocation','file:///C:/Users/Administrator/Documents/Code/Python/PySpark/Autoloaded/Checkpoint/') \
	.start('file:///C:/Users/Administrator/Documents/Code/Python/PySpark/Autoloaded/Data/')
#
```

#### otro youtuber que tiene cosas interesantes de Databricks:
#### REF : https://www.youtube.com/watch?v=-WyQh_ha3hI

# Volumes

In [19]:
#
print(_spark.catalog.currentCatalog())
print(_spark.catalog.getDatabase('default'))
print(_spark.catalog.getDatabase('default').locationUri)
#

spark_catalog
Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/C:/Users/Administrator/Documents/Code/Python/PySpark/spark-warehouse')
file:/C:/Users/Administrator/Documents/Code/Python/PySpark/spark-warehouse


In [20]:
#
from pyspark.sql import functions
#
_spark.range(1).select(functions.current_schema()).show()

+----------------+
|current_schema()|
+----------------+
|         default|
+----------------+



```python

#
## REF : https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-volumes
#
_query = "CREATE EXTERNAL VOLUME IF NOT EXISTS spark_catalog.default.myExternalVolume COMMENT 'This is my example external volume' LOCATION 'file:///C:/Users/Administrator/Documents/Code/Python/PySpark/Volumes/'" ## file:///C:/Users/.../Volumes/parquet/
#
_spark.sql(sqlQuery=_query)
#

```

```python

#
## REF : https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-volumes
#
_query = "SELECT * FROM parquet.'/Volumes/spark_catalog/default/myExternalVolume/parquet'
#
_spark.sql(sqlQuery=_query)
#

```

## Data Versioning & Time Travel

In [ ]:
#
## _query = 'DROP TABLE spark_catalog.default.myTimeTravelTable'
## _spark.sql(sqlQuery=_query)
#

DataFrame[]

In [18]:
_query = "CREATE TABLE IF NOT EXISTS spark_catalog.default.myTimeTravelTable (id INT, name STRING, salary FLOAT) USING DELTA LOCATION 'file:///C:/Users/Administrator/Documents/Code/Python/PySpark/TimeTravelTable'"
_spark.sql(sqlQuery=_query)

DataFrame[]

In [19]:
_query = """INSERT INTO spark_catalog.default.myTimeTravelTable VALUES (0,'a',0.1),(1,'b',0.2),(2,'c',0.3)"""
### print(_query)
_spark.sql(sqlQuery=_query)

DataFrame[]

In [20]:
_query = "DELETE FROM spark_catalog.default.myTimeTravelTable WHERE id = 0"
### print(_query)
_spark.sql(sqlQuery=_query)

DataFrame[num_affected_rows: bigint]

In [23]:
#
from pyspark.sql.functions import col
#
_query = "DESCRIBE HISTORY spark_catalog.default.myTimeTravelTable"
### print(_query)
_dfHistory = _spark.sql(sqlQuery=_query).sort(col('version').asc())
_dfHistory.show(500, truncate=False)
#

+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                            |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                           

In [25]:
_query = "RESTORE TABLE spark_catalog.default.myTimeTravelTable TO VERSION AS OF 1"
### print(_query)
_spark.sql(sqlQuery=_query)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [29]:
_query = "SELECT * FROM spark_catalog.default.myTimeTravelTable ORDER BY id ASC"
### print(_query)
_spark.sql(sqlQuery=_query).show(500, truncate=False)

+---+----+------+
|id |name|salary|
+---+----+------+
|0  |a   |0.1   |
|1  |b   |0.2   |
|2  |c   |0.3   |
+---+----+------+



In [30]:
#
from pyspark.sql.functions import col
#
_query = "DESCRIBE HISTORY spark_catalog.default.myTimeTravelTable"
### print(_query)
_dfHistory = _spark.sql(sqlQuery=_query).sort(col('version').asc())
_dfHistory.show(500, truncate=False)
#

+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                            |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                           

## OPTIMIZE

In [32]:
#
#
_spark.sql(sqlQuery='OPTIMIZE spark_catalog.default.myTimeTravelTable')
#
#
_query = "SELECT * FROM spark_catalog.default.myTimeTravelTable ORDER BY id ASC"
### print(_query)
_spark.sql(sqlQuery=_query).show(500, truncate=False)
#

+---+----+------+
|id |name|salary|
+---+----+------+
|0  |a   |0.1   |
|1  |b   |0.2   |
|2  |c   |0.3   |
+---+----+------+



## 🚀 SQL `OPTIMIZE` Command in PySpark

The `OPTIMIZE` command in PySpark SQL is designed to **enhance performance** and **reduce the overhead** of querying Delta Lake tables by compacting small files.

### 🔧 Purpose
- Merges small data files into larger ones
- Boosts read efficiency and lowers latency
- Ideal for large-scale analytic workloads

### 📌 Syntax
```sql
OPTIMIZE table_name [WHERE predicate]
```
- `table_name`: The Delta Lake table you want to optimize
- `WHERE predicate`: (Optional) Filter rows to optimize specific partitions or data ranges

🧠 Key Features
- Targets **Delta Lake** tables only
- Can be used with **Z-Ordering** to optimize column-specific data layouts

## 🧲 Example
```sql
-- Optimizes entire table
OPTIMIZE sales_data;

-- Optimizes specific partition
OPTIMIZE sales_data WHERE region = 'West';
```

### 📌 Usage
- Applies to Delta Lake tables only
- Optional filtering with `WHERE` clause to optimize specific partitions or subsets of data

### 🧠 Features
- Supports partition-level optimization
- Can be paired with **Z-Ordering** to cluster data by specific columns for improved scan performance

### ⚠️ Notes
- Only usable on tables stored in Delta format
- Most effective in environments with frequent data insertions or streaming workloads


## ZORDERING (liquid clustering)

In [35]:
#
## REF : https://learn.microsoft.com/en-us/azure/databricks/delta/clustering
#
## REF : https://delta.io/blog/liquid-clustering/
#
_spark.sql(sqlQuery='OPTIMIZE spark_catalog.default.myTimeTravelTable ZORDER BY id')
#
#
_query = "SELECT * FROM spark_catalog.default.myTimeTravelTable ORDER BY id ASC"
### print(_query)
_spark.sql(sqlQuery=_query).show(500, truncate=False)
#

+---+----+------+
|id |name|salary|
+---+----+------+
|0  |a   |0.1   |
|1  |b   |0.2   |
|2  |c   |0.3   |
+---+----+------+

